<img src="https://certificate.tpq.io/taim_logo.png" width="350px" align="right">

# Reinforcement Learning

## Finance

Dr Yves J Hilpisch | The AI Machine

http://aimachine.io | http://twitter.com/dyjh

## Imports

In [ ]:
!git clone https://github.com/tpq-classes/rl_for_finance.git
import sys
sys.path.append('rl_for_finance')


In [ ]:
import os
import numpy as np
import pandas as pd
from pylab import plt
pd.set_option('mode.chained_assignment', None)
pd.set_option('display.float_format', '{:.4f}'.format)
np.set_printoptions(suppress=True, precision=4)
os.environ['PYTHONHASHSEED'] = '0'

## Training & Validation

In [ ]:
import finance
import tradingbot

In [ ]:
symbol = 'EUR='
features = [symbol, 'r', 's', 'm', 'v']

In [ ]:
a = 0
b = 1750
c = 250

In [ ]:
learn_env = finance.Finance(symbol, features, window=20, lags=6,
                 leverage=1, min_performance=0.85,
                 start=a, end=a + b, mu=None, std=None)

In [ ]:
learn_env.data.info()

In [ ]:
valid_env = finance.Finance(symbol, features, window=learn_env.window,
                 lags=learn_env.lags, leverage=learn_env.leverage,
                 min_performance=learn_env.min_performance,
                 start=a + b, end=a + b + c,
                 mu=learn_env.mu, std=learn_env.std)

In [ ]:
valid_env.data.info()

In [ ]:
tradingbot.set_seeds(100)
agent = tradingbot.TradingBot(32, 0.00001, learn_env, valid_env)

In [ ]:
episodes = 46

In [ ]:
%time agent.learn(episodes)

In [ ]:
tradingbot.plot_treward(agent)

In [ ]:
tradingbot.plot_performance(agent)

## Vectorized Backtesting

In [ ]:
def reshape(s, env):
    return np.reshape(s, [1, env.lags, env.n_features])

In [ ]:
import tensorflow as tf

In [ ]:
def backtest(agent, env):
    done = False
    env.data['p'] = 0
    state = env.reset()
    while not done:
        action = np.argmax(
            agent.model(tf.convert_to_tensor(reshape(state, env), dtype=tf.float32), training=False).numpy()[0, 0])
        position = 1 if action == 1 else -1
        env.data.loc[:, 'p'].iloc[env.bar] = position
        state, reward, done, info = env.step(action)
    env.data['s'] = env.data['p'] * env.data['r']

In [ ]:
env = agent.learn_env

In [ ]:
%time backtest(agent, env)

In [ ]:
env.data['p'].iloc[env.lags:].value_counts()

In [ ]:
env.data[['r', 's']].iloc[env.lags:].sum().apply(np.exp)

In [ ]:
env.data[['r', 's']].iloc[env.lags:].sum().apply(np.exp) - 1

In [ ]:
env.data[env.symbol].plot(figsize=(10, 6));

In [ ]:
env.data[['r', 's']].iloc[env.lags:].cumsum().apply(np.exp).plot(figsize=(10, 6));

In [ ]:
test_env = finance.Finance(symbol, features, window=learn_env.window,
                 lags=learn_env.lags, leverage=learn_env.leverage,
                 min_performance=learn_env.min_performance,
                 start=a + b + c, end=None,
                 mu=learn_env.mu, std=learn_env.std)

In [ ]:
env = test_env

In [ ]:
backtest(agent, env)

In [ ]:
env.data['p'].iloc[env.lags:].value_counts()

In [ ]:
env.data[['r', 's']].iloc[env.lags:].sum().apply(np.exp)

In [ ]:
env.data[['r', 's']].iloc[env.lags:].sum().apply(np.exp) - 1

In [ ]:
env.data[['r', 's']].iloc[env.lags:].cumsum().apply(np.exp).plot(figsize=(10, 6));

## Event-Based Backtesting

In [ ]:
import backtesting as bt

In [ ]:
bb = bt.BacktestingBase(env=agent.learn_env, model=agent.model,
                       amount=10000, ptc=0.0001, ftc=1.0,
                       verbose=True)

In [ ]:
bb.initial_amount

In [ ]:
bar = 100

In [ ]:
bb.get_date_price(bar)

In [ ]:
bb.env.get_state(bar)

In [ ]:
bb.place_buy_order(bar, amount=5000)

In [ ]:
bb.print_net_wealth(2 * bar)

In [ ]:
bb.place_sell_order(2 * bar, units=1000)

In [ ]:
bb.close_out(3 * bar)

In [ ]:
class TBBacktester(bt.BacktestingBase):
    def _reshape(self, state):
        ''' Helper method to reshape state objects.
        '''
        return np.reshape(state, [1, self.env.lags, self.env.n_features])
    def backtest_strategy(self):
        ''' Event-based backtesting of the trading bot's performance.
        '''
        self.units = 0
        self.position = 0
        self.trades = 0
        self.current_balance = self.initial_amount
        self.net_wealths = list()
        for bar in range(self.env.lags, len(self.env.data)):
            date, price = self.get_date_price(bar)
            if self.trades == 0:
                print(50 * '=')
                print(f'{date} | *** START BACKTEST ***')
                self.print_balance(bar)
                print(50 * '=')
            state = self.env.get_state(bar)
            action = np.argmax(self.model(tf.convert_to_tensor(self._reshape(state.values), dtype=tf.float32), training=False).numpy()[0, 0])
            position = 1 if action == 1 else -1
            if self.position in [0, -1] and position == 1:
                if self.verbose:
                    print(50 * '-')
                    print(f'{date} | *** GOING LONG ***')
                if self.position == -1:
                    self.place_buy_order(bar - 1, units=-self.units)
                self.place_buy_order(bar - 1, amount=self.current_balance)
                if self.verbose:
                    self.print_net_wealth(bar)
                self.position = 1
            elif self.position in [0, 1] and position == -1:
                if self.verbose:
                    print(50 * '-')
                    print(f'{date} | *** GOING SHORT ***')
                if self.position == 1:
                    self.place_sell_order(bar - 1, units=self.units)
                self.place_sell_order(bar - 1, amount=self.current_balance)
                if self.verbose:
                    self.print_net_wealth(bar)
                self.position = -1
            self.net_wealths.append((date, self.calculate_net_wealth(price)))
        self.net_wealths = pd.DataFrame(self.net_wealths,
                                        columns=['date', 'net_wealth'])
        self.net_wealths.set_index('date', inplace=True)
        self.net_wealths.index = pd.DatetimeIndex(self.net_wealths.index)
        self.close_out(bar)

In [ ]:
env = learn_env

In [ ]:
tb = TBBacktester(env, agent.model, 10000, 0.0, 0, verbose=False)

In [ ]:
%time tb.backtest_strategy()

In [ ]:
tb_ = TBBacktester(env, agent.model, 10000, ptc=0.0001, ftc=2.5, verbose=False)

In [ ]:
%time tb_.backtest_strategy()

In [ ]:
ax = tb.net_wealths.plot(figsize=(10, 6))
tb_.net_wealths.columns = ['net_wealth (after tc)']
tb_.net_wealths.plot(ax=ax);

In [ ]:
env = test_env

In [ ]:
tb = TBBacktester(env, agent.model, 10000, 0.0, 0, verbose=False)

In [ ]:
%time tb.backtest_strategy()

In [ ]:
tb_ = TBBacktester(env, agent.model, 10000, 0.0001, 2.5, verbose=False)

In [ ]:
%time tb_.backtest_strategy()

In [ ]:
ax = tb.net_wealths.plot(figsize=(10, 6))
tb_.net_wealths.columns = ['net_wealth (after tc)']
tb_.net_wealths.plot(ax=ax);

In [ ]:
ax = (tb.net_wealths / tb.net_wealths.iloc[0]).plot(figsize=(10, 6))
tp = env.data[['r', 's']].iloc[env.lags:].cumsum().apply(np.exp)
(tp / tp.iloc[0]).plot(ax=ax);

## Average True Range

In [ ]:
data = pd.DataFrame(learn_env.data[symbol])

In [ ]:
data.head()

In [ ]:
window = 14

In [ ]:
data['min'] = data[symbol].rolling(window).min()

In [ ]:
data['max'] = data[symbol].rolling(window).max()

In [ ]:
data['mami'] = data['max'] - data['min']

In [ ]:
data['mac'] = abs(data['max'] - data[symbol].shift(1))

In [ ]:
data['mic'] = abs(data['min'] - data[symbol].shift(1))

In [ ]:
data['atr'] = np.maximum(data['mami'], data['mac'])

In [ ]:
data['atr'] = np.maximum(data['atr'], data['mic'])

In [ ]:
data['atr%'] = data['atr'] / data[symbol]

In [ ]:
data[['atr', 'atr%']].plot(subplots=True, figsize=(10, 6));

In [ ]:
data[['atr', 'atr%']].tail()

In [ ]:
leverage = 10

In [ ]:
data[['atr', 'atr%']].tail() * leverage

In [ ]:
data[['atr', 'atr%']].median() * leverage

## Risk Management

In [ ]:
import tbbacktesterrm as tbbrm

In [ ]:
env = test_env

In [ ]:
tb = tbbrm.TBBacktesterRM(env, agent.model, 10000, 0.0, 0, verbose=False)

In [ ]:
tb.backtest_strategy(sl=None, tsl=None, tp=None, wait=5)

In [ ]:
tb.backtest_strategy(sl=0.017, tsl=None, tp=None, wait=5)

In [ ]:
tb.backtest_strategy(sl=None, tsl=0.02, tp=None, wait=5)

In [ ]:
tb.backtest_strategy(sl=None, tsl=None, tp=0.0125, wait=5)

In [ ]:
tb.backtest_strategy(sl=0.015, tsl=None, tp=0.0185, wait=5)

In [ ]:
tb.backtest_strategy(sl=None, tsl=0.02, tp=0.02, wait=5)

<img src='http://hilpisch.com/taim_logo.png' width="350px" align="right">

<br><br><br><a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:ai@tpq.io">ai@tpq.io</a>